In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

df = pd.read_csv('data.csv')
df['theme'] = df['theme'].apply(lambda x: x.split('_'))

# Step 3: Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Step 4: One-hot encode the themes
encoded_labels = mlb.fit_transform(df['theme'])

# Step 5: Create a DataFrame for the encoded labels
df_encoded = pd.DataFrame(encoded_labels, columns=mlb.classes_)

# Step 6: Convert the one-hot encoded columns back to a list of labels
df['labels'] = df_encoded.values.tolist()

# Step 6: Combine the original dataframe with the encoded labels
df = pd.concat([df, df_encoded], axis=1)

# Step 7: Print the processed DataFrame
print(df.columns)

df.to_csv('data_encoded.csv', index=False)

Index(['sentence', 'aspect', 'theme', 'description', 'ANA', 'BIB', 'DAT',
       'EXP', 'INT', 'MET', 'OAL', 'PDI', 'RES', 'RWK', 'TNF'],
      dtype='object')


In [3]:
# Calculate the frequency of each label
label_counts = df_encoded.sum(axis=0)

# Print the counts for each label
print(label_counts)

ANA     45
BIB     17
DAT    124
EXP    261
INT     13
MET    937
OAL    121
PDI     38
RES    165
RWK    227
TNF    108
dtype: int64
